In [ ]:
%pip install jsonschema networkx numpy scipy pythreejs
%pip install compas>=2.0.0 --no-deps
%pip install compas_notebook --no-deps
%pip install compas_fd>=0.5.0 --no-deps

# Getting Started with COMPAS FD

In [ ]:
from compas.colors import Color
from compas.datastructures import Mesh
from compas_fd.solvers import fd_numpy
from compas_notebook.viewer import Viewer

In [ ]:
mesh = Mesh.from_meshgrid(dx=10, nx=10)

In [ ]:
vertices = mesh.vertices_attributes("xyz")
fixed = list(mesh.vertices_where(vertex_degree=2))
edges = list(mesh.edges())
loads = [[0, 0, 0] for _ in range(len(vertices))]

In [ ]:
q = []
for edge in edges:
    if mesh.is_edge_on_boundary(edge):
        q.append(10)
    else:
        q.append(1.0)

In [ ]:
result = fd_numpy(
    vertices=vertices,
    fixed=fixed,
    edges=edges,
    forcedensities=q,
    loads=loads,
)

In [ ]:
for vertex, attr in mesh.vertices(data=True):
    attr["x"] = result.vertices[vertex, 0]
    attr["y"] = result.vertices[vertex, 1]
    attr["z"] = result.vertices[vertex, 2]

In [ ]:
viewer = Viewer()
viewer.scene.add(mesh, color=Color.from_hex('#cccccc'))
for vertex in fixed:
    viewer.scene.add(mesh.vertex_point(vertex), color=Color.red())
viewer.show()